<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-all-table-structures-and-relations" data-toc-modified-id="Define-all-table-structures-and-relations-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define all table structures and relations</a></span></li><li><span><a href="#Tables" data-toc-modified-id="Tables-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Tables</a></span><ul class="toc-item"><li><span><a href="#regattas" data-toc-modified-id="regattas-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>regattas</a></span></li><li><span><a href="#races" data-toc-modified-id="races-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>races</a></span></li><li><span><a href="#competitors" data-toc-modified-id="competitors-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>competitors</a></span></li><li><span><a href="#windsources-&amp;-wind" data-toc-modified-id="windsources-&amp;-wind-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>windsources &amp; wind</a></span></li><li><span><a href="#legs" data-toc-modified-id="legs-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>legs</a></span></li><li><span><a href="#race_comp" data-toc-modified-id="race_comp-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>race_comp</a></span></li><li><span><a href="#comp_leg" data-toc-modified-id="comp_leg-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>comp_leg</a></span></li><li><span><a href="#positions" data-toc-modified-id="positions-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>positions</a></span></li><li><span><a href="#courses" data-toc-modified-id="courses-2.9"><span class="toc-item-num">2.9&nbsp;&nbsp;</span>courses</a></span></li><li><span><a href="#marks" data-toc-modified-id="marks-2.10"><span class="toc-item-num">2.10&nbsp;&nbsp;</span>marks</a></span></li><li><span><a href="#marks_positions" data-toc-modified-id="marks_positions-2.11"><span class="toc-item-num">2.11&nbsp;&nbsp;</span>marks_positions</a></span></li><li><span><a href="#course_areas" data-toc-modified-id="course_areas-2.12"><span class="toc-item-num">2.12&nbsp;&nbsp;</span>course_areas</a></span></li><li><span><a href="#sailing_style" data-toc-modified-id="sailing_style-2.13"><span class="toc-item-num">2.13&nbsp;&nbsp;</span>sailing_style</a></span></li><li><span><a href="#temp_markpassings" data-toc-modified-id="temp_markpassings-2.14"><span class="toc-item-num">2.14&nbsp;&nbsp;</span>temp_markpassings</a></span></li><li><span><a href="#Temp" data-toc-modified-id="Temp-2.15"><span class="toc-item-num">2.15&nbsp;&nbsp;</span>Temp</a></span></li></ul></li></ul></div>

# Define all table structures and relations

In [11]:
import pyodbc

server = 'sic-match-analysis.database.windows.net'
database = 'match-analysis'
username = 'sic-admin'
password = 'ZeilenIsLeuk!'
driver= '{ODBC Driver 17 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)


cursor = cnxn.cursor()

# Tables

## regattas

In [4]:
# TABLE 1
cursor.execute("""
CREATE TABLE regattas
(regatta varchar(50) PRIMARY KEY,
boatclass varchar(50) NULL,
courseAreaId nvarchar(40) NULL)

CREATE INDEX idx ON regattas (regatta);

""")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'regattas' in the database. (2714) (SQLExecDirectW)")

## races

In [7]:
# TABLE 2
cursor.execute("""
CREATE TABLE races
(race_id binary(16) NOT NULL,
regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL,
raceShort varchar(50) NOT NULL,
fleet varchar(20),
maxWindSpd_kts decimal(10,2) NULL,
minWindSpd_kts decimal(10,2) NULL,
avgWindDir_deg decimal(10,2) NULL,
firstlegbearing_deg decimal(10,2) NULL,
course_area_id int NULL,
nr_competitors int NULL,
[startOfRace-ms] bigint,
[endOfRace-ms] bigint,
start_wind_dir decimal(5,2)
,stl_bearing decimal(5,2)
,stl_bearing_diff_wind decimal(5,2)
,stl_fav_side varchar(10)
,sail_style_id int
)

CREATE UNIQUE CLUSTERED INDEX idx_rac ON races (race_id);
CREATE INDEX idx_reg ON races (regatta);

""")


In [69]:
cursor.execute("""
ALTER TABLE races
   ADD fleet varchar(20)
;
""")
cursor.commit()

In [9]:
# Relation 1
cursor.execute("""
ALTER TABLE races
   ADD CONSTRAINT FK_races_regattas FOREIGN KEY (regatta)
      REFERENCES regattas (regatta)
      ON DELETE CASCADE
      ON UPDATE CASCADE
;
""")
cursor.commit()

## competitors

In [6]:
# TABLE 3
cursor.execute("""
DROP TABLE competitors
CREATE TABLE competitors
(regatta varchar(50) NOT NULL,
comp_id binary(16) NOT NULL,
comp_name varchar(100) NULL,
nationality varchar(3) NULL,
sailId varchar(10) NULL,
[Speed (kts) (Average)] decimal(10,2) NULL,
[Speed (kts) (Average) UPWIND] decimal(10,2) NULL,
[Speed (kts) (Average) DOWNWIND] decimal(10,2) NULL,
[Speed (kts) (Average) REACHING] decimal(10,2) NULL,
[Distance Traveled (Sum)] decimal(10,2) NULL,
[Distance Traveled (Sum) UPWIND] decimal(10,2) NULL,
[Distance Traveled (Sum) DOWNWIND] decimal(10,2) NULL,
[Distance Traveled (Sum) REACHING] decimal(10,2) NULL,
[Speed (kts) (Average) null] decimal(10,2) NULL,
[Distance Traveled (Sum) null] decimal(10,2) NULL,
overall_rank int NULL,
[Number of Maneuvers (Sum)] decimal(10,2) NULL
)

CREATE UNIQUE CLUSTERED INDEX idx ON competitors (regatta, comp_id);
CREATE INDEX idx_comp ON competitors (comp_id)
""")
cursor.commit()

In [9]:
# Relation 2
cursor.execute("""
ALTER TABLE competitors
   ADD CONSTRAINT FK_comp_regattas FOREIGN KEY (regatta)
      REFERENCES regattas (regatta)
      ON DELETE CASCADE
      ON UPDATE CASCADE
;
""")
cursor.commit()

## windsources & wind

In [44]:
cursor.execute(""" 
CREATE TABLE windsources (
race_id binary(16) NOT NULL
,typeName varchar(100) 
,id varchar(50) NOT NULL
,rc bit
)

CREATE UNIQUE CLUSTERED INDEX idx ON windsources (race_id, typeName, id)

ALTER TABLE windsources
   ADD CONSTRAINT FK_windsources_races FOREIGN KEY (race_id)
      REFERENCES races (race_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

In [60]:
cursor.execute(""" 
CREATE TABLE wind (
race_id binary(16) NOT NULL
,windSource varchar(100) NOT NULL
,windSource_id varchar(50) NULL
,[trueBearing-deg] decimal(5,2) NULL
,[speed-kts] decimal(5,2) NULL
,[speed-m/s] decimal(5,2) NULL
,[timepoint-ms] bigint NULL
,[dampenedTrueBearing-deg] decimal(5,2) NULL
,[dampenedSpeed-kts] decimal(5,2) NULL
,[dampenedSpeed-m/s] decimal(5,2) NULL
,[lat-deg] decimal(10,7) NULL
,[lng-deg] decimal(10,7) NULL
)

CREATE UNIQUE CLUSTERED INDEX idx ON wind (race_id, windSource, windSource_id, [timepoint-ms])
CREATE INDEX idx_race On wind (race_id)

ALTER TABLE wind
   ADD CONSTRAINT FK_wind_windsources FOREIGN KEY (race_id, windSource, windSource_id)
      REFERENCES windsources (race_id, typeName, id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")


In [61]:
cursor.commit()

## legs

In [34]:
# TABLE 5
cursor.execute("""
CREATE TABLE legs
(race_id binary(16),
leg_nr int NOT NULL,
fromWaypointId binary(16) NULL,
toWaypointId binary(16) NULL,
[to] varchar(20) NULL,
[from] varchar(20) NULL,
upOrDownwindLeg bit NULL,
leg_nr_from_finish int NULL,
distance decimal(20,2) NULL,
correlation_tacks decimal(5,4),
correlation_side decimal(5,4),
correlation_avgSOG decimal(5,4),
correlation_traveledDistance decimal(5,4),
correlation_jibes decimal(5,4),
pos_startline_abs_x decimal(4,3)
,pos_startline_abs_y decimal(4,3)
,pos_startline_rel decimal(4,3)
,cor_side_pos_startline_abs decimal(4,3)
,cor_side_pos_startline_rel decimal(4,3)
,avg_spd decimal(5,3)
,[avg_distanceTraveled-m] decimal (6,2)
)


CREATE UNIQUE CLUSTERED INDEX idx_ ON legs (race_id, leg_nr);
CREATE INDEX idx_race ON legs (race_id)

""")

In [7]:
cursor.execute("""
ALTER TABLE legs
ADD [avg_distanceTraveled-m] decimal (6,2)

""")

In [8]:
cursor.commit()

In [37]:
# Relation 4

cursor.execute("""
ALTER TABLE legs
   ADD CONSTRAINT FK_legs_races FOREIGN KEY (race_id)
      REFERENCES races (race_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

ProgrammingError: ('42S01', "[42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]There is already an object named 'FK_legs2_races' in the database. (2714) (SQLExecDirectW); [42S01] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Could not create constraint or index. See previous errors. (1750)")

## race_comp

In [30]:
# TABLE 6
cursor.execute("""
DROP TABLE race_comp
CREATE TABLE race_comp
(race_id binary(16)
,comp_id binary(16) NOT NULL
,regatta varchar(50) NOT NULL
,rank int
,pos_startline_abs_x decimal(3, 2)
,pos_startline_abs_y decimal(7, 5)
,pos_startline_rel decimal(4, 2) NULL)

CREATE UNIQUE CLUSTERED INDEX idx ON race_comp (race_id, comp_id);
CREATE INDEX idx_race ON race_comp (race_id);
CREATE INDEX idx_comp ON race_comp (comp_id);

""")

In [41]:
cursor.commit()

In [31]:
# Relation 5&6
cursor.execute("""
ALTER TABLE race_comp
   ADD CONSTRAINT FK_race_comp_races FOREIGN KEY (race_id)
      REFERENCES races (race_id),
    CONSTRAINT FK_race_comp_competitors FOREIGN KEY(regatta, comp_id)
      REFERENCES competitors (regatta, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

## comp_leg

In [39]:
# TABLE 7
cursor.execute("""
CREATE TABLE comp_leg
(race_id binary(16)
,leg_nr int NOT NULL,
comp_id binary(16) NOT NULL,
[competitor_distanceTraveled-m] decimal(10,2) NULL,
[competitor_averageSOG-kts] decimal(5,2) NULL,
competitor_tacks int NULL,
competitor_jibes int NULL,
competitor_penaltyCircles int NULL,
competitor_rank int NULL,
[competitor_gapToLeader-s] decimal(10,2) NULL,
[competitor_gapToLeader-m] decimal(10,2) NULL,
competitor_started bit NULL,
competitor_finished bit NULL,
avg_side decimal(5,2) NULL,
most_left decimal(5,2) NULL,
most_right decimal(5,2) NULL,
rel_rank decimal(3,2) NULL,
rel_averageSOG decimal (5,3) NULL, 
rel_distanceTraveled decimal(5,3) NULL
)

CREATE UNIQUE CLUSTERED INDEX idx ON comp_leg (race_id, leg_nr, comp_id);
CREATE INDEX idx_rac_com ON comp_leg (race_id, comp_id);
CREATE INDEX idx_leg ON comp_leg (race_id, leg_nr);

""")

In [9]:
cursor.execute("""
ALTER TABLE comp_leg
ADD rel_averageSOG decimal (5,3) NULL


""")

In [10]:
cursor.commit()

In [40]:
# Relation 7&8

cursor.execute("""
ALTER TABLE comp_leg
   ADD CONSTRAINT FK_comp_leg_race_comp FOREIGN KEY (race_id, comp_id)
      REFERENCES race_comp (race_id, comp_id),
      CONSTRAINT FK_comp_leg_legs FOREIGN KEY (race_id, leg_nr)
      REFERENCES legs (race_id, leg_nr)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

## positions

In [55]:
# TABLE 8
cursor.execute("""
CREATE TABLE positions
(race_id binary(16) NOT NULL
,leg_nr int NULL,
comp_id binary(16) NOT NULL,
timepoint_ms bigint NOT NULL,
[lat-deg] decimal(20,10) NULL,
[lng-deg] decimal(20,10) NULL,
[speed-kts] decimal(10,2) NULL,
[truebearing-deg] decimal(10,2) NULL,
calculated_windSpd decimal(10,2) NULL,
calculated_windDir decimal(10,2) NULL,
[rel_spd-kts] decimal(5,3)
)

CREATE UNIQUE CLUSTERED INDEX idx ON positions (race_id, leg_nr, comp_id, timepoint_ms ASC);
CREATE INDEX idx_legnr ON positions (leg_nr) 
CREATE INDEX idx_leg ON positions (race_id, leg_nr, comp_id);
CREATE INDEX idx_comp ON positions (race_id, comp_id)
""")

In [56]:
# Relation 9
cursor.execute("""
ALTER TABLE positions
   ADD CONSTRAINT FK_positions_race_comp FOREIGN KEY (race_id,  comp_id)
      REFERENCES race_comp (race_id,  comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE,
   CONSTRAINT FK_positions_comp_leg FOREIGN KEY (race_id, leg_nr, comp_id)
     REFERENCES comp_leg (race_id, leg_nr, comp_id)
""")

## courses

In [49]:
cursor.execute("""
CREATE TABLE courses (
race_id binary(16) NOT NULL
,name varchar(50) NULL
,passingInstruction varchar(50) NULL
,[controlPoint.@class] varchar(50) NULL
,[controlPoint.name] varchar(50) NULL
,[controlPoint.id] binary(16) NULL
,[controlPoint.left.@class]varchar(50) NULL
,[controlPoint.left.name] varchar(50) NULL
,[controlPoint.left.id] binary(16)NULL
,[controlPoint.left.type] varchar(50) NULL
,[controlPoint.right.@class]varchar(50) NULL
,[controlPoint.right.name] varchar(50) NULL
,[controlPoint.right.id] binary(16) NULL
,[controlPoint.right.type] varchar(50) NULL
,[controlPoint.type] varchar(50) NULL
,mark_nr int NOT NULL
,mark_nr_from_finish int NULL)

CREATE UNIQUE CLUSTERED INDEX idx ON courses (race_id, mark_nr)
CREATE INDEX idx_race ON courses (race_id)


""")

In [50]:
cursor.execute("""
ALTER TABLE courses
   ADD CONSTRAINT FK_courses_races FOREIGN KEY (race_id)
      REFERENCES races (race_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
;
""")

In [66]:
cursor.commit()

## marks

In [46]:
cursor.execute("""
CREATE TABLE marks (
race_id binary(16) NOT NULL
,id binary(16) NOT NULL
,name varchar(50) NULL
)

CREATE UNIQUE CLUSTERED INDEX idx ON marks (race_id, id)
CREATE INDEX idx_race ON marks (race_id)


""")

In [47]:
# Relation 1

cursor.execute("""
ALTER TABLE marks
   ADD CONSTRAINT FK_marks_races FOREIGN KEY (race_id)
      REFERENCES races (race_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
;""")

## marks_positions

In [64]:
# TABLE 11
cursor.execute("""
CREATE TABLE marks_positions
(race_id binary(16) NOT NULL,
id binary(16) NOT NULL,
[lat-deg] decimal(10,7) NULL,
[lng-deg] decimal(10,7) NULL,
[timepoint-ms] bigint,
)



CREATE UNIQUE CLUSTERED INDEX idx ON marks_positions (race_id, id, [timepoint-ms]);
CREATE INDEX idx_mrks ON marks_positions (race_id, id)

""")


In [65]:
cursor.execute("""

ALTER TABLE marks_positions
   ADD CONSTRAINT FK_marks_positions_marks FOREIGN KEY (race_id,  id)
      REFERENCES marks (race_id,  id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
;""")

## course_areas

In [3]:
cursor.execute("""
CREATE TABLE course_areas (
	id int,
    course_area varchar(20),
    [lat-deg] decimal(10,7),
    [lng-deg] decimal(10,7)
    );

CREATE UNIQUE INDEX idx ON course_areas (id)

INSERT INTO course_areas(id, course_area, lat, lon)
VALUES (1, 'Enoshima', 35.1756, 139.2951),
(2, 'Fujisawa',35.1583, 139.2920),
(3, 'Kamakura', 35.1744, 139.3079),
(4, 'Sagami', 35.1513, 139.3103),
(5, 'Zushi', 35.1621, 139.3239),
(6, 'Hayama', 35.1441, 139.3285)
""")

In [53]:
cursor.execute("""
ALTER TABLE races
ADD CONSTRAINT fk_races_course_areas FOREIGN KEY (course_area_id)
      REFERENCES course_areas (id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

## sailing_style

In [ ]:
## create table with types and id's

cursor.execute("""
CREATE TABLE sailing_style
(id int NOT NULL
,style varchar(50) NOT NULL
,tacks_lower decimal(3,2)
,tacks_upper decimal(3,2)
,side_lower decimal(3,2)
,side_upper decimal(3,2)
,speed_lower decimal(3,2)
,speed_upper decimal(3,2)
,distance_lower decimal(3,2)
,distance_upper decimal(3,2)
)

AlTER TABLE races
ADD sail_style_id int



CREATE UNIQUE CLUSTERED INDEX idx ON sailing_style (id);



""")



In [ ]:
cursor.execute("""
INSERT INTO sailing_style
VALUES (1, 'Oscillating', -1, .1, 0, .2, -.1, 0, .5, 1),
(2, 'Persistent', .5, 1, .5, 1, -.2, .2, .5, 1),
(3, 'Geographical', -.2, .2, .5, 1, -1, -.3, .3, -1),
(4, 'Connect Pressure', -.5, 0, 0,.2, -1, -.8, -1, 0),
(5, 'Random', 0,0, 0, 0, 0, 0, 0, 0)
""")



## temp_markpassings

In [ ]:
### Markpassings, to update legnr_columnn in positions

cursor.execute("""
CREATE TABLE temp_markpassings
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL
,race_id binary(16)
comp_id binar(16) NOT NULL,
leg_nr int NOT NULL,
begin_leg_ms bigint NULL,
end_leg_ms bigint NULL

)

CREATE UNIQUE INDEX idx ON temp_markpassings (regatta, race, leg_nr, comp_id);
CREATE CLUSTERED INDEX idx_2 ON temp_markpassings (regatta, race, comp_id);

""")

In [ ]:
cursor.execute("""
ALTER TABLE temp_markpassings
ADD CONSTRAINT fk_temp_markpassings_comp_leg FOREIGN KEY (regatta, race, leg_nr, comp_id)
      REFERENCES comp_leg (regatta, race, leg_nr, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

In [ ]:
### Markpassings, to update legnr_columnn in positions

cursor.execute("""
DROP TABLE temp_markpassings
CREATE TABLE temp_markpassings
(regatta varchar(50) NOT NULL,
race varchar(50) NOT NULL
,race_id binary(16)
comp_id binary(16) NOT NULL,
leg_nr int NOT NULL,
begin_leg_ms bigint NULL,
end_leg_ms bigint NULL

)

CREATE UNIQUE INDEX idx ON temp_markpassings (regatta, race, leg_nr, comp_id);
CREATE CLUSTERED INDEX idx_2 ON temp_markpassings (regatta, race, comp_id);

""")

cursor.execute("""
ALTER TABLE temp_markpassings
ADD CONSTRAINT fk_temp_markpassings_comp_leg FOREIGN KEY (regatta, race, leg_nr, comp_id)
      REFERENCES comp_leg (regatta, race, leg_nr, comp_id)
      ON DELETE CASCADE
      ON UPDATE CASCADE
""")

In [15]:
cnxn.commit()

## Temp


In [ ]:
sql = """SELECT *
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_NAME = N'Customers'"""


In [ ]:
cursor.execute(sql)
cursor.commit()